In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json, urllib2
import os, glob, sys, time
import nltk
%matplotlib inline

### Helper functions

In [3]:
#Read 70k recipes from allrecipes.com into a list of dictionaries
def read_data():
    folder='recipes/'
    files=pd.read_csv('all_files.txt',header=None)[0].values
    k=0
    recipes=[]
    st=time.time()
    for filename in files:
        f=open(folder+filename,'r')
        r=json.load(f)
        recipes.append(r)
        k+=1
        if k%10000==0:
            print k
    return recipes

#Removing and replacing some noizy symbols
def clean_string(s):    
    sep_symbols=[';']
    for ss in sep_symbols:
        s=s.replace(ss,'.')
    for i in range(10):
        s=s.replace('..','.')
    bad_symbols=[')','(','!','-']
    for bs in bad_symbols:
        s=s.replace(bs,' ')
    s=s.replace(',',' , ')                  
    s=s.replace('  ',' ')
    s=s.replace('. ','.')
    return s

#Raw direction text -> List of single directions
def get_clean_directions(recipe):
    raw=recipe['directions']
    direction=''
    for dd in raw:
        direction=direction+dd+'.'
    direction=clean_string(direction).lower()
    s=direction.split('.')
    return s

### Highlighting recipes

In [7]:
def bright(l,l_idx):
    l2=[]
    for i in range(len(l)):
        if l_idx[i]:
            l2.append(yellow(l[i]))
        else:
            l2.append(l[i])
    l2=' '.join(l2)
    return l2

def yellow(s):
    return '\x1b[43m'+s+'\x1b[0m'

def green(s):
    return '\x1b[32m'+s+'\x1b[0m'

def highlight_recipe(recipes,recipe_id):
    dirs=get_clean_directions(recipes[recipe_id])
    for d in dirs:
        if len(d)>0:
            d_words=np.array(d.split(' '))
            action_idx=np.array([(word in actions_set) for word in d_words])
            print bright(d_words,action_idx)

### Main

##### Loading data

In [12]:
%time recipes=read_data()
actions=pd.read_csv('actions_dict_sorted.txt',sep=' ')

10000
20000
30000
40000
50000
60000
70000
Wall time: 7.05 s


#### Printing results

In [13]:
actions_set=set(actions[:100].word.values)

In [14]:
highlight_recipe(recipes,0)

combine parmesan cheese , pepper and garlic powder
unfold pastry sheets onto cutting board
brush lightly with egg white
sprinkle each sheet with 1/4 of the cheese mixture
lightly press into pastry , turn over
repeat
cut each sheet into 12 1 inch strips
twist
place on ungreased cookie sheet and bake in 350 degrees f 175 degrees c oven for 15 minutes or until golden brown
